In [123]:
# Imports first
import csv
import time
import math
import geopandas as gpd
import pandas as pd
import shapely
from shapely.ops import nearest_points, transform
import numpy as np
from scipy import spatial
import pyproj
from mpl_toolkits.basemap import Basemap

In [73]:
# Set notebook display options
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [74]:
def find_nearest(row, dest_df=None, geom1_col='geometry', geom2_col='geometry'):
    if dest_df is None:
        raise ValueError('Destination DataFrame (df2) argument not provided')
    D = spatial.distance_matrix([[row[geom1_col].x, row[geom1_col].y]],
                                [[pt.x, pt.y] for pt in dest_df[geom2_col]])
    nn = np.array([[np.min(D[0,]), np.argmin(D[0,])]])
    distance = float(nn[:,0][0])
    nearest_index = int(nn[:,1][0])
    return nearest_index, distance

In [75]:
def geod2utm(row):
    '''   Convert geodetic coordinates to UTM   '''
    zn = '16T'
    datum = 'WGS84'
    lat = row['lat']
    lon = row['lon']
        
    p = pyproj.Proj(proj='utm', zone=zn, ellps=datum)
    
    X, Y = p(lon, lat)
    
    return pd.Series({'UTMx': X, 'UTMy': Y})

In [76]:
def make_utm_points(row):
    UTMx = row['UTMx']
    UTMy = row['UTMy']
    UTMPoint = shapely.geometry.Point(UTMx, UTMy)
    return UTMPoint

In [77]:
def assign_category(row):
    categories: dict = {"THEFT": "property",
                    "BURGLARY": "property",
                    "MOTOR VEHICLE THEFT": "property",
                    "ARSON": "property",
                    "CRIMINAL DAMAGE": "property",
                    "ROBBERY": "property",
                    "ASSAULT": "person",
                    "BATTERY": "person",
                    "CRIM SEXUAL ASSAULT": "person",
                    "HOMICIDE": "person",
                    "INTIMIDATION": "person",
                    "KIDNAPPING": "person",
                    "OFFENSE INVOLVING CHILDREN": "person",
                    "SEX OFFENSE": "person",
                    "STALKING": "person",
                    "GAMBLING": "vice",
                    "NARCOTICS": "vice",
                    "PROSTITUTION": "vice",
                    "LIQUOR LAW VIOLATION": "vice",
                    "OBSCENITY": "vice",
                    "OTHER NARCOTIC VIOLATION": "vice",
                    "PUBLIC INDECENCY": "vice",
                    "OTHER OFFENSE": "other",
                    "DECEPTIVE PRACTICE": "other",
                    "WEAPONS VIOLATION": "other",
                    "PUBLIC PEACE VIOLATION": "other",
                    "CRIMINAL TRESPASS": "other",
                    "INTERFERENCE WITH PUBLIC OFFICER": "other",
                    "NON-CRIMINAL": "other"
                   }
    return categories[row['primary type']]

In [78]:
def get_bearing(row: pd.Series, point2: shapely.geometry.Point, point1_geom='geometry'):
    """A method that returns the number of degrees from due north a line is.

    This line is defined as being from point1 (typically the source point) 
    to point2 (typically the destination).
    
    The point point1 is presumed to come from the row indexed by the string
    'geometry' in the pandas Series passed to this method as row.

    This function is designed to be used in a pandas.DataFrame.apply() call.
    """
    
    point1:shapely.geometry.Point = row[point1_geom]
    point3:shapely.geometry.Point = shapely.geometry.Point(point1.x, point2.y)
    A:float = point1.distance(point3) # adjacent leg
    B:float = point2.distance(point3) # opposite leg
    C:float = point1.distance(point2) # hypotenuse leg
    # To find the bearing, we need to know the angle from a right triangle
    # defined by point1 (the source point), point2 (the destination point), and
    # point3, the point due north or south of point1 at a distance defined by
    # point2.y.
    #
    # The angle needed is of the leg between point1 and point3 and the leg
    # between point1 and point2.
    theta:float = math.degrees(math.asin(A/C))
    # Start with bearing of 0 degrees (a.k.a. due north)
    bearing:float = 0.0
    # X is equal, so point 2 is either north or south
    if point2.x == point1.x:
        # if point2.y is bigger, it's north, so no need to modify bearing
        # otherwise, bearing is 180 degrees
        if point2.y < point1.y:
            bearing = 180.0
    # Y is equal, so either due east or due west
    elif point2.y == point1.y:
        # point2.x is bigger, so due west
        if point2.x > point1.x:
            bearing = 270
        # point2.x is smaller, so due east
        else:
            bearing = 90
    # Point2.x is bigger, so Point2 is west of Point1
    elif point2.x > point1.x:
        # Point2.y is bigger, so NW
        if point2.y > point1.y:
            bearing = theta + 270
        # Point2.y is smaller, so SW
        else:
            bearing = theta + 180
    # Point2.x is smaller, so Point2 is east of Point1
    else:
        # Point2.y is bigger, so NE
        if point2.y > point1.y:
            bearing = theta
            # bearing = theta + 90
        # Point2.y is smaller, so SE
        else:
            bearing = theta + 90
    return round(bearing, 2)

In [79]:
def get_community(row: pd.Series,
                  community_areas: pd.DataFrame,
                  location_point: shapely.geometry.Point,
                  geom2_column: str='UTMGeometry'
                 ):
    community_id = community_areas.apply(lambda community:
                                         location_point.within(community[geom2_column]))
    return community_id

# Enhance crimes data
* Identify nearest school, library, and police station, and the distance and compass heading to each
* Summarize the report type into one of four categories:
  * Property: crimes involving the stealing of or damage to property
  * Person: crimes against a human, such as assault, homicide, or kidnapping
  * Vice: violations of morality-based laws, such as prostitution, gambling, or drug/alcohol
  * Other: anything reported not in the above three categories

In [9]:
categories: dict = {"THEFT": "property",
                    "BURGLARY": "property",
                    "MOTOR VEHICLE THEFT": "property",
                    "ARSON": "property",
                    "CRIMINAL DAMAGE": "property",
                    "ROBBERY": "property",
                    "ASSAULT": "person",
                    "BATTERY": "person",
                    "CRIM SEXUAL ASSAULT": "person",
                    "HOMICIDE": "person",
                    "INTIMIDATION": "person",
                    "KIDNAPPING": "person",
                    "OFFENSE INVOLVING CHILDREN": "person",
                    "SEX OFFENSE": "person",
                    "STALKING": "person",
                    "GAMBLING": "vice",
                    "NARCOTICS": "vice",
                    "PROSTITUTION": "vice",
                    "LIQUOR LAW VIOLATION": "vice",
                    "OBSCENITY": "vice",
                    "OTHER NARCOTIC VIOLATION": "vice",
                    "PUBLIC INDECENCY": "vice",
                    "OTHER OFFENSE": "other",
                    "DECEPTIVE PRACTICE": "other",
                    "WEAPONS VIOLATION": "other",
                    "PUBLIC PEACE VIOLATION": "other",
                    "CRIMINAL TRESPASS": "other",
                    "INTERFERENCE WITH PUBLIC OFFICER": "other",
                    "NON-CRIMINAL": "other"
                   }

In [10]:
crimes_df = pd.read_pickle("data/crimes-transformed.pkl")

In [11]:
crimes_df['category'] = crimes_df.apply(assign_category, axis=1)

In [12]:
crimes = gpd.GeoDataFrame(crimes_df, geometry='UTMPoint')

In [13]:
crimes.crs = {'init' :'epsg:2966'}

In [14]:
# Create a small extract of data to test performance of algorithms
crimes_extract = crimes.iloc[0:100]

In [15]:
schools_df = pd.read_pickle("data/schools-transformed.pkl")

In [16]:
neighborhoods = pd.read_pickle("data/neighborhoods.pkl")

In [17]:
schools_df.set_index("UNIT_ID")
schools_df['UNIT_ID'] = schools_df.index

In [18]:
schools = gpd.GeoDataFrame(schools_df)

In [19]:
schools_unary_union = schools.unary_union

In [21]:
start_time = time.time()
crimes = pd.concat([crimes, pd.DataFrame(crimes.apply(find_nearest,
                                                      dest_df=schools,
                                                      geom1_col='UTMPoint',
                                                      geom2_col='geometry',
                                                      axis=1
                                                     ).tolist(),
                                        columns=['nearest_school_id', 'nearest_school_distance'],
                                        index=crimes.index)], axis=1)
end_time = time.time()
print("That took {0} seconds".format(end_time - start_time))

That took 7957.16069316864 seconds


# Enhance crimes with identity and distance to nearest police station

In [22]:
police_stations_df = pd.read_pickle('data/police-stations-transformed.pkl')
police_stations = gpd.GeoDataFrame(police_stations_df)

In [23]:
police_stations = police_stations.set_geometry('UTMPoint')

In [24]:
start_time = time.time()
crimes = pd.concat([crimes, pd.DataFrame(crimes.apply(find_nearest,
                                                      dest_df=police_stations,
                                                      geom1_col='UTMPoint',
                                                      geom2_col='UTMPoint',
                                                      axis=1
                                                     ).tolist(),
                                        columns=['nearest_station_id', 'nearest_station_distance'],
                                        index=crimes.index)], axis=1)
end_time = time.time()
print("That took {0} seconds".format(end_time - start_time))

That took 375.9948363304138 seconds


# Enhance crimes with identity and distance to nearest library

In [25]:
libraries_df = None
libraries = None
libraries_df = pd.read_pickle('data/libraries-transformed.pkl')

In [26]:
libraries = gpd.GeoDataFrame(libraries_df)
libraries = libraries.set_geometry('UTMPoint')

In [27]:
start_time = time.time()
crimes = pd.concat([crimes, pd.DataFrame(crimes.apply(find_nearest,
                                                      dest_df=libraries,
                                                      geom1_col='UTMPoint',
                                                      geom2_col='UTMPoint',
                                                      axis=1
                                                     ).tolist(),
                                        columns=['nearest_library_id', 'nearest_library_distance'],
                                        index=crimes.index)], axis=1)
end_time = time.time()
print("That took {0} seconds".format(end_time - start_time))

That took 1063.915910243988 seconds


In [28]:
crimes['nearest_library_bearing'] = crimes.apply(lambda row: get_bearing(row,
                                     point2=libraries.iloc[row['nearest_library_id']][libraries.geometry.name],
                                     point1_geom=crimes.geometry.name), axis=1)

In [29]:
crimes['nearest_school_bearing'] = crimes.apply(lambda row: get_bearing(row,
                                     point2=schools.iloc[row['nearest_school_id']][schools.geometry.name],
                                     point1_geom=crimes.geometry.name), axis=1)

In [30]:
crimes['nearest_station_bearing'] = crimes.apply(lambda row: get_bearing(row,
                                     point2=police_stations.iloc[row['nearest_station_id']][police_stations.geometry.name],
                                     point1_geom=crimes.geometry.name), axis=1)

In [64]:
crimes['community area'] = crimes['community area'].astype(dtype='int64')
crimes['ward'] = crimes['ward'].astype(dtype='Int64')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
crimes.fillna(0)

In [82]:
crimes.to_pickle('data/crimes-enhanced.pkl')

# Enhance crimes with community area information
Todo: comment out the read of crimes for subsequent runs of the whole notebook

In [80]:
# crimes = pd.read_pickle('data/crimes-enhanced.pkl')
census = pd.read_pickle('data/census-summary-enhanced.pkl')

In [90]:
crimes.groupby('community area')['community area'].sum().sort_values(ascending=False)

community area
71    1484468
67    1476479
66    1350492
68    1324436
69    1261596
25    1174100
43    1049071
49     993426
61     943426
44     651376
73     634808
53     629375
46     622886
29     604418
28     544992
77     539308
23     528241
24     515616
42     511812
32     494048
63     483651
70     462350
75     454875
58     419862
30     368940
27     360882
38     357390
65     354055
22     340692
26     328952
40     322320
76     291232
56     287672
60     286380
19     274569
51     261681
31     215078
48     207024
35     196945
52     193336
8      182376
72     181440
41     180031
62     178622
64     175040
59     171808
45     170415
39     170274
54     157896
57     150537
21     149331
16     141264
15     135345
50     128100
33     124146
74     121582
34      95472
14      92932
6       89880
20      89500
37      88874
55      87285
7       77532
17      76602
36      59436
47      42911
18      31482
13      29939
11      29656
3       29532
10   

In [124]:
m = Basemap(width=1,height=1, resolution='l',projection='laea',lat_ts=50,lat_0=50,lon_0=-107.)
Basemap

In [126]:
projected_geometry = transform(m,census[census['GeogKey'] == 71]['UTMGeometry'].values.item())
projected_geometry.area

0.0

In [130]:
side1 = abs(census[census['GeogKey'] == 71]['UTMGeometry'].values.item().bounds[0] - 
            census[census['GeogKey'] == 71]['UTMGeometry'].values.item().bounds[2]
           )
side2 = abs(census[census['GeogKey'] == 71]['UTMGeometry'].values.item().bounds[1] - 
            census[census['GeogKey'] == 71]['UTMGeometry'].values.item().bounds[3]
           )
print("Side 1: {0}; Side 2: {1}".format(side1, side2))
print(side1 * side2 / 1000)

Side 1: 3705.4170308914618; Side 2: 3274.1869035074487
12132.22791457828


In [146]:
poly71 = census[census['GeogKey'] == 71]['UTMGeometry'].values.item()
poly09 = census[census['GeogKey'] == 9]['UTMGeometry'].values.item()
pop71 = census[census['GeogKey'] == 71]['Total Population'].values.item()
pop09 = census[census['GeogKey'] == 9]['Total Population'].values.item()
density71 = pop71 / poly71.area
density09 = pop09 / poly09.area

In [152]:
#census.apply(lambda row: row['Total Population'].values.item() / row['UTMGeometry'].values.item().area)
census['Total Population'] / census['UTMGeometry'].values.item().area

ValueError: can only convert an array of size 1 to a Python scalar

In [154]:
census.apply(lambda row: row['UTMGeometry'].values.item().area)

KeyError: ('UTMGeometry', 'occurred at index Geog')